In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pyarrow.parquet as pq
from numpy.linalg import norm
import random
import time
from sklearn.svm import SVR

In [ ]:
import os

_REP_PATH_ = os.getcwd()

In [ ]:
# Initialisation des DataFrames contenant les données

# DataFrame des données d'entrainement
train_df = pd.read_csv(_REP_PATH_+'/data/train_nettoye.csv', index_col = 0)

# DataFrame des données de test
test_df = pd.read_csv(_REP_PATH_+'/data/test_nettoye.csv', index_col = 0)


# Affichage des premiers individus des DataFrames
train_df.head()

In [4]:
test_df.head()

,row_ID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee,PU_location_lat,PU_location_lon,DO_location_lat,DO_location_lon
0,0,2,2024-01-24 08:02:47,2024-01-24 08:04:45,1.0,0.36,1.0,N,1,4.4,0.0,0.5,0.0,1.0,2.5,0.0,40.740439,-73.976495,40.747746,-73.978492
1,1,2,2024-01-26 14:34:58,2024-01-26 14:42:51,1.0,1.78,1.0,N,1,10.7,0.0,0.5,0.0,1.0,2.5,0.0,40.791705,-73.973049,40.790011,-73.945750
2,2,2,2024-01-24 15:47:27,2024-01-24 15:51:48,2.0,0.43,1.0,N,2,5.8,0.0,0.5,0.0,1.0,0.0,0.0,40.791705,-73.973049,40.797962,-73.968168
3,3,2,2024-01-04 14:40:17,2024-01-04 14:44:21,1.0,0.61,1.0,N,1,5.8,0.0,0.5,0.0,1.0,2.5,0.0,40.783961,-73.978632,40.791705,-73.973049
4,4,2,2024-01-16 05:54:03,2024-01-16 05:55:59,1.0,0.71,1.0,N,2,5.1,1.0,0.5,0.0,1.0,2.5,0.0,40.748427,-73.999918,40.748427,-73.999918


In [13]:
X_train = train_df[['passenger_count', 'trip_distance', 'RatecodeID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'Airport_fee', 'PU_location_lat', 'PU_location_lon', 'DO_location_lat', 'DO_location_lon']]
X_test = test_df[['passenger_count', 'trip_distance', 'RatecodeID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'Airport_fee', 'PU_location_lat', 'PU_location_lon', 'DO_location_lat', 'DO_location_lon']]

Y_train = train_df['tip_amount']

X_train.head()

# SVR avec noyau gaussien codée à la main

In [14]:
def optimise_alpha(X, Y, Kernel, lbda=1):
    # On transforme nos DataFrame en matrices numpy
    if type(X) == pd.core.frame.DataFrame:
        X = X.to_numpy()
    if type(Y) == pd.core.frame.DataFrame:
        Y = Y.to_numpy()
        
    K = np.array([ [Kernel_gauss(x1,x2) for x2 in X] for x1 in X]) 
    n = len(X)
    alpha = np.linalg.inv(K + len(X)*lbda*np.eye(len(X)))@Y
    return alpha

In [15]:
def predicteur(Kernel, alpha, X_connu, x_predict):
    return np.sum([alpha[i]*Kernel(X_connu[i], x_predict) for i in range(len(X_connu))])

# Noyau Gaussien

In [ ]:
liste_var = [1,10,np.var(Y_train_choice), 20, 30, 40,60,80,200,400,1000,10000]
liste_lbda = [0.00001,0.0001,0.001,0.1,1,2]
liste_lbda = np.linspace(0,0.1,10) # liste_lbda est un tableau
tab_r2_lbda = np.zeros((len(liste_var), len(liste_lbda)))
start = time.time()

for i in range(len(liste_var)):
    var = liste_var[i]
    def Kernel_gauss(x1,x2, sigma2=var):
        return np.exp(-norm(x1-x2)**2/(2*sigma2))
    for j in range(len(liste_lbda)):
        lbda = liste_lbda[j]
        try :
            alpha = optimise_alpha(X_train_choice, Y_train_choice, Kernel_gauss, lbda = lbda)
            Y_pred = [predicteur2(Kernel_gauss, alpha, X_train_choice, x_test) for x_test in X_test_choice]
            tab_r2_lbda[i,j] = r2_score(Y_test_choice, Y_pred)
        except : 
            tab_r2_lbda[i,j] = -1000
        print(" dernier effectué : i = " + str(i) + " / " + str(len(liste_var)) + "   j = " + str(j) + " / " + str(len(liste_lbda)), end = '\r')
    plt.plot(liste_lbda,tab_r2_lbda[i,:], label = 'var = ' +str(var))
print("temps d'exécution  " + str(time.time() - start))
plt.legend()
plt.ylim([-1,1])
plt.xlabel("lambda")
plt.ylabel("score R2")
#plt.title('score R2 pour différentes valeurs de lambda et \sigma**2')
plt.show()

# Noyau Polynomial

In [ ]:
liste_C = [0.01,1,10,10**2,10**3,2*10**3,10**4,10**5]
liste_degree = [1,2,3,4]
tab_r2 = np.zeros((len(liste_degree),len(liste_C)))
start = time.time()
for i in range(len(liste_degree)):
    degree = liste_degree[i]
    for j in range(len(liste_C)):
        C = liste_C[j]
        svr_poly = SVR(kernel='poly', C=C, degree=degree)
        model = svr_poly.fit(X_train_choice,Y_train_choice)
        Y_pred = model.predict(X_test_choice)
        tab_r2[i,j] = r2_score(Y_test_choice, Y_pred)
        print(" dernier effectué : i = " + str(i) + " / " + str(len(liste_degree)) + " j = " + str(j) + " / " + str(len(liste_C)), end = '\r')      
    plt.plot(np.log(liste_C), tab_r2[i,:], label = 'degree ' + str(degree) )
print("temps d'exécution  " + str(time.time() - start))
plt.legend()
plt.ylim([-1,1])
plt.xlabel("hyperparamètre log_10(C)")
plt.ylabel("score R2")
plt.show()

# Prediction avec Noyau Polynomial de degré 2, C = 1000

In [ ]:
indices_selectionnes_train = random.choices(range(len(X_train)), k = 10000)
X_train_reduit = X_train.to_numpy()[indices_selectionnes_train]
Y_train_reduit = Y_train.to_numpy()[indices_selectionnes_train]

In [ ]:
C = 1000
degree = 2
start = time.time()
svr_poly = SVR(kernel='poly', C=C, degree=degree)
model = svr_poly.fit(X_train_reduit,Y_train_reduit)
Y_pred = model.predict(X_test.to_numpy())
print("temps d'exécution  " + str(time.time() - start))
# 32 sec pour k = 1000
# 300 sec pour k = 10 000
# 3700 sec pour X_train entier

In [ ]:
prediction_SVRdegree2 = pd.DataFrame(np.arange(len(Y_pred), dtype = int).T, columns  = ["row_ID"])
prediction_SVRdegree2["tip_amount"] = Y_pred
prediction_SVRdegree2.to_parquet(_REP_PATH_ + "output/prediction_SVRdegree2.parquet")